In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import stats
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain 
from sklearn.metrics import f1_score, jaccard_score


!pip install scikit-multilearn

from skmultilearn.cluster import MatrixLabelSpaceClusterer
from sklearn.cluster import KMeans
from skmultilearn.ensemble import  LabelSpacePartitioningClassifier
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.4 MB/s 


In [ ]:
os.getcwd()
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
from house1_data import *
# from house2_data import *
# from house3_data import *
# from house4_data import *
# from house5_data import *
# from house6_data import *

Import lists:

In [ ]:
appliancesData = redd_house1_appliances
appliancesList = redd_house1_names
targetAppsIndices = redd_house1_targetAppsIndices

Define a bunch of functions:

In [ ]:
""" MEDIAN FILTER """     
def median(aggregate_vector):
    window = 50
    s = pd.Series(aggregate_vector)
    median_df = s.rolling(window).median()
    return median_df.fillna(0)

def plotApp():
    for i in range(0,len(appliancesData)):
        print("{} : {}\n".format(appliancesList[i], i))
    choice = int(input("choose number"))
    plt.plot(appliancesData[choice])
    
def plot_energy_break(appliancesData, appliancesList):
    """ Create a matrix containing the data of each appliance """
    appliances_mtrx=np.zeros((len(appliancesData[0]),
                              len(appliancesData)))
    for i in range(0,len(appliancesData)):
        appliances_mtrx[:,i]=appliancesData[i]
    energy_vector = np.zeros((1,len(appliancesData))) 
    energy_vector = np.sum(appliances_mtrx, axis=0)
    x = np.arange(len(appliancesData))
    plt.bar(x, energy_vector)  
    plt.xticks(x, appliancesList)

def label_load (appliance_serie):
    thr = 50
    fr2 = appliance_serie
    fr3 = fr2.copy()
    fr3 = fr3.to_numpy()
    fr3[fr3 < thr] = 0
    fr3[fr3 >= thr] = 1
    label_vector = fr3
    return label_vector

def label_load_adaptive (appliance_serie):
    thr = float(stats.mode(appliance_serie.to_numpy())[0])+5
    fr2 = appliance_serie
    fr3 = fr2.copy()
    fr3 = fr3.to_numpy()
    fr3[fr3 < thr] = 0
    fr3[fr3 >= thr] = 1
    label_vector = fr3
    return label_vector

def agg_vector(appliancesData):
    """ Create a matrix containing the data of each appliance """
    appliances_mtrx=np.zeros((len(appliancesData[0]),
                              len(appliancesData)))
    for i in range(0,len(appliancesData)):
        appliances_mtrx[:,i]=appliancesData[i]
        
    aggregate_vector = np.zeros((appliances_mtrx.shape[0],1))
    aggregate_vector = np.sum(appliances_mtrx, axis=1) 
    return aggregate_vector 

def agg_vector_only_targets(appliancesData, targetAppsIndices):
    """ Create a matrix containing the data of each appliance """
    appliances_mtrx=np.zeros((len(appliancesData[0]),
                              len(targetAppsIndices)))
    for i in range(0,len(targetAppsIndices)):
        appliances_mtrx[:,i]=median(appliancesData[ targetAppsIndices[i] ]).to_numpy()
        
    aggregate_vector = np.zeros((appliances_mtrx.shape[0],1))
    aggregate_vector = np.sum(appliances_mtrx, axis=1) 
    return aggregate_vector

def agg_vector_median(appliancesData):
    """ Create a matrix containing the data of each appliance """
    appliances_mtrx=np.zeros((len(appliancesData[0]),
                              len(appliancesData)))
    for i in range(0,len(appliancesData)):
        appliances_mtrx[:,i]=median(appliancesData[i]).to_numpy()
        
    aggregate_vector = np.zeros((appliances_mtrx.shape[0],1))
    aggregate_vector = np.sum(appliances_mtrx, axis=1) 
    return aggregate_vector 

def checkLabeling():    
    print("choose load:")
    for i in range(0,len(targetAppsIndices)):
        print("{} : {}\n".format(appliancesList[targetAppsIndices[i]], targetAppsIndices[i]))
    choice = int(input("choose number"))
    plt.plot(median(appliancesData[choice]))
    column_index = np.where(targetAppsIndices_arr==choice)[0]
    plt.plot(label_mtrx[:,column_index]*100)
    plt.show()

""" Dicretization: transform the dataset with KBinsDiscretizer """ 
def discret(aggregate_vector): 
    enc = KBinsDiscretizer(n_bins=100, encode='ordinal', strategy='uniform')
    X_binned = enc.fit_transform(aggregate_vector.reshape(-1,1))
    
    aggregate_vector_denoised = enc.inverse_transform(X_binned)
    return aggregate_vector_denoised

""" Compute absolute changes """ 
def changes(aggregate_vector_denoised):
    aggregate_vector_denoised_x = np.insert(aggregate_vector_denoised,0,0) 
    aggregate_vector_denoised_x_plus = np.insert(aggregate_vector_denoised,
                                            len(aggregate_vector_denoised),0)
    delta_x_vector = aggregate_vector_denoised_x_plus - aggregate_vector_denoised_x
    return delta_x_vector

""" Remove spikes from denoised signal """ 
def remove_spikes (noised_vector, delta_x):
    aggregate_vector_denoised_corrected = np.zeros((len(noised_vector), 1))
    delta_x_vector_corrected = np.zeros((len(delta_x), 1))
    for i in range (1, len(noised_vector)):
        if(delta_x[i]!=0):
            if(delta_x[i+1]==0):
                delta_x_vector_corrected[i] = delta_x[i]
                aggregate_vector_denoised_corrected[i] = noised_vector[i]
            else:
                delta_x_vector_corrected[i] = delta_x[i-1]
                delta_x_vector_corrected[i+1] = delta_x[i-1]
                aggregate_vector_denoised_corrected[i] = noised_vector[i-1]
        if(delta_x[i]==0):
            delta_x_vector_corrected[i] = delta_x[i]
            aggregate_vector_denoised_corrected[i] = noised_vector[i]
    return aggregate_vector_denoised_corrected, delta_x_vector_corrected

""" ************************* remove short events """  
def rmv_evnts(agrregate_final1, delta_x_final1):
    duration_thresold = 5
    events_index = np.where(delta_x_final1!=0)[0]
    
    """ find short events indices """ 
    short_events_index= []
    for i in range(0, len(events_index)-1):
        if(events_index[i+1]-events_index[i]<=duration_thresold):
            duration = events_index[i+1]-events_index[i]
            short_events_index.append((events_index[i],duration))
    
    """ remove the events """ 
    aggregate_final = np.copy(agrregate_final1)
    delta_x_final = np.copy(delta_x_final1)
    for i in range(0, len(short_events_index)-1):
        delta_x_final[short_events_index[i][0]] = 0
        # delta_x_final[short_events_index[i+1]] = 0
        aggregate_final[short_events_index[i][0]:short_events_index[i][0]+short_events_index[i][1]] = aggregate_final[short_events_index[i][0]-1]
      
    """ new delta_x """ 
    aggregate_final_x = np.insert(aggregate_final,0,0) 
    aggregate_final_x_plus = np.insert(aggregate_final,
                                            len(aggregate_final),0)
    delta_x_final = aggregate_final_x_plus - aggregate_final_x  
    
    return aggregate_final, delta_x_final

""" ******************* extract (delta_x, aggregate, duration) """
def features (aggregate_final, delta_x_final, label_mtrx):
    events_indices = np.where(delta_x_final!=0)[0]
    delta_p = []
    state_p =[]
    state_duration = []
    label =[]
    
    for i in range(0, len(events_indices)-1):
        delta_p.append(delta_x_final[events_indices[i]])
        state_p.append(aggregate_final[events_indices[i]])
        state_duration.append(events_indices[i+1] - events_indices[i])
        label.append(label_mtrx[events_indices[i],:])
        
    delta_p = np.array(delta_p)
    delta_p = delta_p.reshape(len(delta_p),1)   
    state_p = np.array(state_p)
    state_p = state_p.reshape(len(state_p),1)   
    state_duration = np.array(state_duration)
    state_duration = state_duration.reshape(len(state_duration),1) 
    """ don't forget """ 
    X = np.concatenate((delta_p,state_p,state_duration), axis=1) 
    # X = np.concatenate((delta_p,state_duration), axis=1)
    Y = np.array(label)
    
    return X, Y

def train_test (classifier, X_train, X_test):
    if (classifier=='ovr'):
        ovr_rf = OneVsRestClassifier(RandomForestClassifier(random_state=0)).fit(
    X_train, Y_train)
        Y_pred_train = ovr_rf.predict_proba(X_train)
        Y_pred_test = ovr_rf.predict_proba(X_test)
        
    if (classifier=='chains'):
        chains = [ClassifierChain(RandomForestClassifier(random_state=0), order='random', random_state=i)
          for i in range(10)]
        for chain in chains:
            chain.fit(X_train, Y_train)
        
        Y_pred_chains = np.array([chain.predict(X_train) for chain in
                          chains])
        Y_pred_ensemble = Y_pred_chains.mean(axis=0)
        Y_pred_train = Y_pred_ensemble 
        
        Y_pred_chains = np.array([chain.predict(X_test) for chain in
                  chains])
        Y_pred_ensemble = Y_pred_chains.mean(axis=0)
        Y_pred_test = Y_pred_ensemble 
        
    
    if (classifier=='mlknn'):
        from skmultilearn.adapt import MLkNN

        classifier = MLkNN(k=71, s=0.7)
        
        # train
        classifier.fit(X_train, Y_train)
        
        # predict
        Y_pred_train = classifier.predict(X_train)
        Y_pred_test = classifier.predict(X_test)
        
    if (classifier=='lsp'):
        
        matrix_clusterer = MatrixLabelSpaceClusterer(clusterer=KMeans(n_clusters=2))
        
        matrix_clusterer.fit_predict(X_train, Y_train)
        
        classifier = LabelSpacePartitioningClassifier(
            classifier = LabelPowerset(classifier=GaussianNB()),
            clusterer = matrix_clusterer
        )
        
        classifier.fit(X_train, Y_train)
                
        # predict
        Y_pred_train = classifier.predict(X_train)
        Y_pred_test = classifier.predict(X_test)
                
    return Y_pred_train, Y_pred_test


Some data manipulation:

In [ ]:
appliancesList_arr = np.array(appliancesList)
targetAppsNames = appliancesList_arr[targetAppsIndices] 
targetAppsNames_arr = targetAppsNames.reshape((1,len(targetAppsNames)))
targetAppsIndices_arr = np.array(targetAppsIndices)
allIndices= np.arange(0,len(appliancesList))
outlierAppsIndices = np.setdiff1d(allIndices, targetAppsIndices)
outliersNames =   np.array(appliancesList)[outlierAppsIndices]

number_of_targets = len(targetAppsIndices)
print("number of target loads: {}".format(number_of_targets))
number_of_outliers = len(appliancesList) - number_of_targets 
print("number of non-targeted loads: {}".format(number_of_outliers))

number of target loads: 7
number of non-targeted loads: 2


Create the labels matrix:

In [ ]:
label_mtrx= np.zeros((len(appliancesData[0]), (len(targetAppsIndices_arr))))

for i in range(0, len(targetAppsIndices_arr)):
    singlaAppData= appliancesData[ targetAppsIndices_arr[i] ]
    # singlaAppData_arr = singlaAppData.to_numpy()
    labelVector = label_load_adaptive(median(singlaAppData))
    # labelVector_column = labelVector.reshape((1,len(labelVector)))
    label_mtrx[:,i] = labelVector

Create the aggregate vector:

In [ ]:
agg_vector_raw = agg_vector(appliancesData)
    
# agg_vector = agg_vector_median(appliancesData)
agg_vector = agg_vector_only_targets(appliancesData,targetAppsIndices)

Discretization of the aggregate vector:

In [ ]:
agg_vector_disc = discret(agg_vector)

Compute absolute changes of the aggregate signal:

In [ ]:
delta_x_vector = changes(agg_vector_disc)

Remove spikes from the aggregate signal:

In [ ]:
agg_vector_filt_1, delta_x_vector_corrected_1 = remove_spikes(agg_vector_disc, delta_x_vector)

Remove short events from the aggregate signal:

In [ ]:
agg_vector_filt, delta_x_vector_corrected  = rmv_evnts(agg_vector_filt_1,
                                                       delta_x_vector_corrected_1)

Feature extraction:

In [ ]:
X, Y = features (agg_vector_filt, delta_x_vector_corrected, label_mtrx)

Splitting into training and testing sets:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8, 
                                                    random_state=4)

Train/test the model:

In [ ]:
Y_pred_train, Y_pred_test = train_test('chains', X_train, X_test)

Model's performance evaluation:

In [ ]:
micro_f1 = f1_score(Y_train, Y_pred_train>0.5, average='micro')
                     
macro_f1 = f1_score(Y_train, Y_pred_train>0.5, average='macro') 
jaccard_score_ = jaccard_score(Y_train, Y_pred_train >= .5, average='samples')
error_training= 1-jaccard_score_

print("training performance")
print("micro-F1: %s"%micro_f1)
print("macro-F1: %s"%macro_f1)
print("training error: %s"%error_training)
print("")

""" ******************************* test on test set """

"""  evauation """

 
micro_f1 = f1_score(Y_test, Y_pred_test>0.5, average='micro')
                
macro_f1 = f1_score(Y_test, Y_pred_test>0.5, average='macro') 
jaccard_score_ = jaccard_score(Y_test, Y_pred_test >= .5, average='samples')
error_testng= 1-jaccard_score_
performance_test=np.mean([micro_f1,macro_f1])
print("test performance")
print("micro-F1: %s"%micro_f1)
print("macro-F1: %s"%macro_f1)
print("Performance: %s"%performance_test)
print("testing error: %s"%error_testng)
print("")

training performance
micro-F1: 0.9994722955145119
macro-F1: 0.9989572471324296
training error: 0.06894484412470026

test performance
micro-F1: 0.8739384288747346
macro-F1: 0.7160122097762324
Performance: 0.7949753193254835
testing error: 0.2082915810208975



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix:

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
conf_mtrx_train =multilabel_confusion_matrix(Y_train, Y_pred_train>0.5)
conf_mtrx_test =multilabel_confusion_matrix(Y_test, Y_pred_test>0.5)

dic ={}
keys = range(len(targetAppsNames))
values = targetAppsNames
for i in keys:
    dic[i] = values[i]

def f_score_train(column):

    # tn = conf_mtrx_train[column, 0, 0]
    tp = conf_mtrx_train[column, 1, 1]
    fn = conf_mtrx_train[column, 1, 0]
    fp = conf_mtrx_train[column, 0, 1]
    f1 = (2*tp)/((2*tp)+fp+fn)
    return f1

def f_score_test(column):

    # tn = conf_mtrx_test[column, 0, 0]
    tp = conf_mtrx_test[column, 1, 1]
    fn = conf_mtrx_test[column, 1, 0]
    fp = conf_mtrx_test[column, 0, 1]
    f1 = (2*tp)/((2*tp)+fp+fn)
    return f1

print("TRAIN")
for i in dic:
    print ("{0} F-score : {1}".format(dic[i],f_score_train(int(i))))
print("")
print("TEST")
for i in dic:
    print ("{0} F-score : {1}".format(dic[i],f_score_test(int(i))))
print("")

TRAIN
oven F-score : 1.0
fridge F-score : 1.0
dish.W F-score : 1.0
lights F-score : 1.0
washing.M F-score : 1.0
mic F-score : 0.9927007299270073
stove F-score : 1.0

TEST
oven F-score : 0.15053763440860216
fridge F-score : 0.8615847542627884
dish.W F-score : 0.5836298932384342
lights F-score : 0.9581514762516046
washing.M F-score : 0.9262295081967213
mic F-score : 0.737527114967462
stove F-score : 0.794425087108014



Number of unique combinations of the different loads:

In [ ]:
#unique_rows_total = np.unique(Y, axis=0)
unique_rows_train = np.unique(Y_train, axis=0)
unique_rows_test = np.unique(Y_test, axis=0)
#print("Number of unique combinantions in total: {}".format(len(unique_rows_total)))
print("Number of unique combinantions in training: {}".format(len(unique_rows_train)))
print("Number of unique combinantions in testing: {}".format(len(unique_rows_test)))
print("")

Number of unique combinantions in training: 29
Number of unique combinantions in testing: 43

